In [ ]:
!pip install mplsoccer
import mysql.connector as connection
import pandas as pd
import matplotlib.pyplot as plt
from mplsoccer import Pitch, VerticalPitch, Radar, FontManager, grid
import os
import panel as pn
import matplotlib.patches as mpatches

pn.extension()

# ALL MATCHES

In [ ]:
try:
    mydb = connection.connect(host="localhost", database='mydb',user="root", passwd="",use_pure=True)
    query = "SELECT match_id, match_date, t1.team_name as home_team_name, t2.team_name as away_team_name FROM mydb.Match INNER JOIN Team t1 ON home_team_id=t1.team_id INNER JOIN Team t2 ON away_team_id=t2.team_id WHERE match_competition_id=%s AND match_season_id=%s;"
    df_match = pd.read_sql(query,mydb, params=[43, 106])
    mydb.close() #close the connection
except Exception as e:
    mydb.close()
    print(str(e))

In [ ]:
df_match
df_match.sort_values(by='match_date', ascending = True, inplace = True)

In [ ]:
df_match['match_name'] = df_match['home_team_name'] + ' - ' + df_match['away_team_name']
df_match.sort_values(by='match_date', ascending = True, inplace = True) 


match_dict_home = df_match.groupby(['home_team_name'])['match_id', 'match_name', 'match_date'].apply(lambda grp : grp.to_dict(orient='records')).to_dict()
match_dict_away = df_match.groupby(['away_team_name'])['match_id', 'match_name', 'match_date'].apply(lambda grp : grp.to_dict(orient='records')).to_dict()
import bisect
for team, matches in match_dict_away.items():
    for match in matches:
        bisect.insort(match_dict_home[team], match, key=lambda x: x['match_date'])
    #match_dict_home[team].extend(matches)
#match_dict_home['Argentina']

In [ ]:
match_list = df_match.match_id
match_list_id = match_list.to_list()
df_team_match = df_match.loc[df_match['match_id'].isin(match_list_id)][['match_id', 'home_team_name', 'away_team_name', 'match_name']]
match_name_list = df_match.loc[df_match['match_id'].isin(match_list_id)][['match_id', 'home_team_name', 'away_team_name', 'match_name']].match_name
match_name_list.tolist()

match_name_dict = {}
for index, row in df_team_match.iterrows():
    match_name_dict[row['match_name']] = row['match_id']

# PLAYER ROLE

In [ ]:
def findPlayerRole(player_name):
    try:
        mydb = connection.connect(host="localhost", database='mydb',user="root", passwd="",use_pure=True)
        query = "SELECT player_name, position_name FROM Event INNER JOIN Position ON Event.position_id=Position.position_id INNER JOIN Player ON Event.player_id=Player.player_id WHERE Player.player_name=%s;"
        df_events = pd.read_sql(query,mydb, params=[player_name])
        return df_events.value_counts().index[0][1]
        mydb.close() #close the connection
    except Exception as e:
        mydb.close()
        print(str(e))




## Minutes Played

In [ ]:
def getMinutesPlayed(player_name, team_name):
    try:
        mydb = connection.connect(host="localhost", database='mydb',user="root", passwd="",use_pure=True)
        dict_id_lambda = list(map(lambda x: x['match_id'], match_dict_home[team_name]))
        mins_tot = 0
        for i in dict_id_lambda:

            query = "SELECT E.match_id, S.type_name as type_name, E.period, E.minute, P1.player_name as player_name, P2.player_name as substitution_replacement_name, Fc.foul_committed_card_name, Bb.bad_behaviour_card_name FROM Event E LEFT JOIN Substitution S ON E.id=S.id INNER JOIN Player P1 ON E.player_id=P1.player_id LEFT JOIN Player P2 ON S.substitution_replacement_id=P2.player_id LEFT JOIN FoulCommitted Fc ON E.id=Fc.id LEFT JOIN BadBehaviour Bb ON E.id=Bb.id WHERE E.position_id!=26 AND (P1.player_name=%s OR P2.player_name=%s) AND E.match_id=%s;"
            df_event = pd.read_sql(query,mydb, params=[player_name, player_name, i])
            if df_event['player_name'].str.contains(player_name).any() == False:
                continue
            else:
            #giocatore subentrato
                if (len(df_event[(df_event['type_name'] == 'Substitution') & (df_event['substitution_replacement_name'] == player_name)])==1) & (len(df_event[(df_event['type_name'] == 'Substitution') & (df_event['player_name'] == player_name)])==0):
                    if len(df_event[(df_event['type_name'] == 'Substitution') & (df_event['substitution_replacement_name'] == player_name) & ((df_event['period']==1) | (df_event['period']==2))])==1:
                        if int(df_event[(df_event['type_name'] == 'Substitution') & (df_event['substitution_replacement_name'] == player_name)].minute)<90:
                            mins = 90 - int(df_event[(df_event['type_name'] == 'Substitution') & (df_event['substitution_replacement_name'] == player_name)].minute)
                        else:
                            mins = df_event['minute'].max() - int(df_event[(df_event['type_name'] == 'Substitution') & (df_event['substitution_replacement_name'] == player_name)].minute)
                    
                    if len(df_event[(df_event['type_name'] == 'Substitution') & (df_event['substitution_replacement_name'] == player_name) & ((df_event['period']==3) | (df_event['period']==4))])==1:
                        if int(df_event[(df_event['type_name'] == 'Substitution') & (df_event['substitution_replacement_name'] == player_name)].minute)<120:
                            mins = 120 - int(df_event[(df_event['type_name'] == 'Substitution') & (df_event['substitution_replacement_name'] == player_name)].minute)
                        else:
                            mins = df_event['minute'].max() - int(df_event[(df_event['type_name'] == 'Substitution') & (df_event['substitution_replacement_name'] == player_name)].minute)
                        
                    if 'bad_behaviour_card_name' in df_event.columns:
                        if len(df_event[((df_event['bad_behaviour_card_name'] == 'Second Yellow') | (df_event['bad_behaviour_card_name'] == 'Red Card')) & (df_event['player_name'] == player_name)])==1:
                            mins = int(df_event[((df_event['bad_behaviour_card_name'] == 'Second Yellow') | (df_event['bad_behaviour_card_name'] == 'Red Card')) & (df_event['player_name'] == player_name)].minute) - int(df_event[(df_event['type_name'] == 'Substitution') & (df_event['substitution_replacement_name'] == player_name)].minute)
                    if 'foul_committed_card_name' in df_event.columns:
                        if len(df_event[((df_event['foul_committed_card_name'] == 'Second Yellow') | (df_event['foul_committed_card_name'] == 'Red Card')) & (df_event['player_name'] == player_name)])==1:
                            mins = int(df_event[((df_event['foul_committed_card_name'] == 'Second Yellow') | (df_event['foul_committed_card_name'] == 'Red Card')) & (df_event['player_name'] == player_name)].minute) - int(df_event[(df_event['type_name'] == 'Substitution') & (df_event['substitution_replacement_name'] == player_name)].minute)
                #giocatore sostituito da titolare
                if (len(df_event[(df_event['type_name'] == 'Substitution') & (df_event['substitution_replacement_name'] == player_name)])==0) & (len(df_event[(df_event['type_name'] == 'Substitution') & (df_event['player_name'] == player_name)])==1):
                    mins = int(df_event[(df_event['type_name'] == 'Substitution') & (df_event['player_name'] == player_name)].minute)

                #giocatore sostituito dopo essere subentrato
                if (len(df_event[(df_event['type_name'] == 'Substitution') & (df_event['substitution_replacement_name'] == player_name)])==1) & (len(df_event[(df_event['type_name'] == 'Substitution') & (df_event['player_name'] == player_name)])==1):
                    mins = int(df_event[(df_event['type_name'] == 'Substitution') & (df_event['player_name'] == player_name)].minute) - int(df_event[(df_event['type_name'] == 'Substitution') & (df_event['substitution_replacement_name'] == player_name)].minute)



                #giocatore sempre in campo
                if len(df_event[(df_event['type_name'] == 'Substitution') & ((df_event['player_name'] == player_name) | (df_event['substitution_replacement_name'] == player_name))])==0:
                    if (df_event['period'].max())==2:
                        mins = 90
                    if (df_event['period'].max())==4 or (df_event['period'].max())==5:
                        #print('ok')
                        mins = 120
                    if 'bad_behaviour_card_name' in df_event.columns:
                        if len(df_event[((df_event['bad_behaviour_card_name'] == 'Second Yellow') | (df_event['bad_behaviour_card_name'] == 'Red Card')) & (df_event['player_name'] == player_name)])==1:
                            mins = int(df_event[((df_event['bad_behaviour_card_name'] == 'Second Yellow') | (df_event['bad_behaviour_card_name'] == 'Red Card')) & (df_event['player_name'] == player_name)].minute)
                    if 'foul_committed_card_name' in df_event.columns:
                        if len(df_event[((df_event['foul_committed_card_name'] == 'Second Yellow') | (df_event['foul_committed_card_name'] == 'Red Card')) & (df_event['player_name'] == player_name)])==1:
                            mins = int(df_event[((df_event['foul_committed_card_name'] == 'Second Yellow') | (df_event['foul_committed_card_name'] == 'Red Card')) & (df_event['player_name'] == player_name)].minute)
                mins_tot = mins_tot + mins
        mydb.close() #close the connection
        return mins_tot
        
    except Exception as e:
        mydb.close()
        print(str(e))

## xG p90

In [ ]:
def getxGValuePer90(player_name, team_name):
    try:
        player_min = getMinutesPlayed(player_name, team_name)/90
        mydb = connection.connect(host="localhost", database='mydb',user="root", passwd="",use_pure=True)
        query = "SELECT shot_statsbomb_xg FROM Event INNER JOIN Shot ON Event.id=Shot.id INNER JOIN Player ON Event.player_id=Player.player_id WHERE Event.period!=5 AND Player.player_name=%s;"
        df_xG = pd.read_sql(query,mydb, params=[player_name])
        mydb.close() #close the connection
        if player_min==0:
            return 0
        else:
            return (df_xG['shot_statsbomb_xg'].sum())/player_min
        
    except Exception as e:
        mydb.close()
        print(str(e))

## Shots p90

In [ ]:
def getShotsValuePer90(player_name, team_name):
    try:
        player_min = getMinutesPlayed(player_name, team_name)/90
        mydb = connection.connect(host="localhost", database='mydb',user="root", passwd="",use_pure=True)
        query = "SELECT * FROM Event INNER JOIN Shot ON Event.id=Shot.id INNER JOIN Player ON Event.player_id=Player.player_id WHERE Player.player_name=%s;"
        df_shots = pd.read_sql(query,mydb, params=[player_name])
        mydb.close() #close the connection
        if player_min==0:
            return 0
        else:
            return len(df_shots)/player_min
        
    except Exception as e:
        mydb.close()
        print(str(e))

## xG/Shot

In [ ]:
def getxGperShotValue(player_name, team_name):
    player_shots = getShotsValuePer90(player_name, team_name)
    player_xg = getxGValuePer90(player_name, team_name)
    if player_shots == 0:
        return 0.0
    else:
        player_xgpershot = player_xg/player_shots
    
    return player_xgpershot

## Dribbles Completed p90

In [ ]:
def getDribblesCompletedPer90(player_name, team_name):
    try:
        player_min = getMinutesPlayed(player_name, team_name)/90
        mydb = connection.connect(host="localhost", database='mydb',user="root", passwd="",use_pure=True)
        query = "SELECT player_name, Dribble.type_name FROM Event INNER JOIN Dribble ON Event.id=Dribble.id INNER JOIN Player ON Event.player_id=Player.player_id WHERE Dribble.outcome_id=8 and Player.player_name=%s;"
        df_dr_compl = pd.read_sql(query,mydb, params=[player_name])
        mydb.close() #close the connection
        if player_min==0:
            return 0
        else:
            return len(df_dr_compl)/player_min
        
    except Exception as e:
        mydb.close()
        print(str(e))

## Pressures p90

In [ ]:
def getPressuresPer90(player_name, team_name):
    try:
        player_min = getMinutesPlayed(player_name, team_name)/90
        mydb = connection.connect(host="localhost", database='mydb',user="root", passwd="",use_pure=True)
        query = "SELECT player_name, Pressure.type_name FROM Event INNER JOIN Pressure ON Event.id=Pressure.id INNER JOIN Player ON Event.player_id=Player.player_id WHERE Player.player_name=%s;"
        df_pressures = pd.read_sql(query,mydb, params=[player_name])
        mydb.close() #close the connection
        if player_min==0:
            return 0
        else:
            return len(df_pressures)/player_min
        
    except Exception as e:
        mydb.close()
        print(str(e))

## Dispossessions p90

In [ ]:
def getDispossessionsPer90(player_name, team_name):
    try:
        player_min = getMinutesPlayed(player_name, team_name)/90
        mydb = connection.connect(host="localhost", database='mydb',user="root", passwd="",use_pure=True)
        query = "SELECT player_name, Dispossessed.type_name FROM Event INNER JOIN Dispossessed ON Event.id=Dispossessed.id INNER JOIN Player ON Event.player_id=Player.player_id WHERE Player.player_name=%s;"
        df_disp = pd.read_sql(query,mydb, params=[player_name])
        mydb.close() #close the connection
        if player_min==0:
            return 0
        else:
            return len(df_disp)/player_min
       
    except Exception as e:
        mydb.close()
        print(str(e))

## Passing Percentage

In [ ]:
def getPassingPercentage(player_name):
    try:
        mydb = connection.connect(host="localhost", database='mydb',user="root", passwd="",use_pure=True)
        query1 = "SELECT player_name, Pass.type_name FROM Event INNER JOIN Pass ON Event.id=Pass.id INNER JOIN Player ON Event.player_id=Player.player_id WHERE Pass.outcome_id IS NULL AND Player.player_name=%s;"
        df_passes_compl = pd.read_sql(query1,mydb, params=[player_name])
        query2 = "SELECT player_name, Pass.type_name FROM Event INNER JOIN Pass ON Event.id=Pass.id INNER JOIN Player ON Event.player_id=Player.player_id WHERE Pass.outcome_id IS NOT NULL AND Player.player_name=%s;"
        df_passes_notcompl = pd.read_sql(query2,mydb, params=[player_name])
        mydb.close() #close the connection
        if (len(df_passes_compl)+len(df_passes_notcompl)) == 0:
            return 0.0
        else:
            perc = len(df_passes_compl)/(len(df_passes_compl)+len(df_passes_notcompl))*100
            return perc
        
    except Exception as e:
        mydb.close()
        print(str(e))


## Fouls Won p90

In [ ]:
def getFoulsWonPer90(player_name, team_name):
    try:
        player_min = getMinutesPlayed(player_name, team_name)/90
        mydb = connection.connect(host="localhost", database='mydb',user="root", passwd="",use_pure=True)
        query = "SELECT player_name, FoulWon.type_name FROM Event INNER JOIN FoulWon ON Event.id=FoulWon.id INNER JOIN Player ON Event.player_id=Player.player_id WHERE Player.player_name=%s;"
        df_foulwon = pd.read_sql(query,mydb, params=[player_name])
        mydb.close() #close the connection
        if player_min==0:
            return 0
        else:
            return len(df_foulwon)/player_min
        
    except Exception as e:
        mydb.close()
        print(str(e))

## Crosses p90

In [ ]:
def getCrossesPer90(player_name, team_name):
    try:
        player_min = getMinutesPlayed(player_name, team_name)/90
        mydb = connection.connect(host="localhost", database='mydb',user="root", passwd="",use_pure=True)
        query = "SELECT player_name, Pass.type_name FROM Event INNER JOIN Pass ON Event.id=Pass.id INNER JOIN Player ON Event.player_id=Player.player_id WHERE Pass.pass_cross=1 AND Player.player_name=%s;"
        df_crosses = pd.read_sql(query,mydb, params=[player_name])
        mydb.close() #close the connection
        if player_min==0:
            return 0
        else:
            return len(df_crosses)/player_min
        
    except Exception as e:
        mydb.close()
        print(str(e))

## Crosses Percentage

In [ ]:
def getCrossesPercentage(player_name):
    try:
        mydb = connection.connect(host="localhost", database='mydb',user="root", passwd="",use_pure=True)
        query1 = "SELECT player_name, Pass.type_name FROM Event INNER JOIN Pass ON Event.id=Pass.id INNER JOIN Player ON Event.player_id=Player.player_id WHERE Pass.outcome_id IS NULL AND Pass.pass_cross=1 AND Player.player_name=%s;"
        df_crosses_compl = pd.read_sql(query1,mydb, params=[player_name])
        query2 = "SELECT player_name, Pass.type_name FROM Event INNER JOIN Pass ON Event.id=Pass.id INNER JOIN Player ON Event.player_id=Player.player_id WHERE Pass.outcome_id IS NOT NULL AND Pass.pass_cross=1 AND Player.player_name=%s;"
        df_crosses_notcompl = pd.read_sql(query2,mydb, params=[player_name])
        mydb.close() #close the connection
        if (len(df_crosses_compl)+len(df_crosses_notcompl))==0:
            return 0.0
        else:
            perc = len(df_crosses_compl)/(len(df_crosses_compl)+len(df_crosses_notcompl))*100
            return perc
        
    except Exception as e:
        mydb.close()
        print(str(e))

## Fouls Committed p90

In [ ]:
def getFoulsCommittedPer90(player_name, team_name):
    try:
        player_min = getMinutesPlayed(player_name, team_name)/90
        mydb = connection.connect(host="localhost", database='mydb',user="root", passwd="",use_pure=True)
        query = "SELECT player_name, FoulCommitted.type_name, FoulCommitted.foul_committed_card_name FROM Event INNER JOIN FoulCommitted ON Event.id=FoulCommitted.id INNER JOIN Player ON Event.player_id=Player.player_id WHERE Player.player_name=%s;"
        df_foulcomm = pd.read_sql(query,mydb, params=[player_name])
        mydb.close() #close the connection
        if player_min==0:
            return 0
        else:
            return len(df_foulcomm)/player_min
        
    except Exception as e:
        mydb.close()
        print(str(e))

## Carries p90

In [ ]:
def getCarriesPer90(player_name, team_name):    
    try:
        player_min = getMinutesPlayed(player_name, team_name)/90
        mydb = connection.connect(host="localhost", database='mydb',user="root", passwd="",use_pure=True)
        query = "SELECT player_name, Carry.type_name FROM Event INNER JOIN Carry ON Event.id=Carry.id INNER JOIN Player ON Event.player_id=Player.player_id WHERE Player.player_name=%s;"
        df_carries = pd.read_sql(query,mydb, params=[player_name])
        mydb.close() #close the connection
        if player_min==0:
            return 0
        else:
            return len(df_carries)/player_min
        
    except Exception as e:
        mydb.close()
        print(str(e))

## Clearances p90

In [ ]:
def getClearancesPer90(player_name, team_name):
    try:
        player_min = getMinutesPlayed(player_name, team_name)/90
        mydb = connection.connect(host="localhost", database='mydb',user="root", passwd="",use_pure=True)
        query = "SELECT player_name, Clearance.type_name FROM Event INNER JOIN Clearance ON Event.id=Clearance.id INNER JOIN Player ON Event.player_id=Player.player_id WHERE Player.player_name=%s;"
        df_clearances = pd.read_sql(query,mydb, params=[player_name])
        mydb.close() #close the connection
        if player_min==0:
            return 0
        else:
            return len(df_clearances)/player_min
        
    except Exception as e:
        mydb.close()
        print(str(e))

## Duels Won Percentage

In [ ]:
def getDuelsWonPercentage(player_name):
    try:
        mydb = connection.connect(host="localhost", database='mydb',user="root", passwd="",use_pure=True)
        query1 = "SELECT player_name, Duel.type_name FROM Event INNER JOIN Duel ON Event.id=Duel.id INNER JOIN Player ON Event.player_id=Player.player_id WHERE (Duel.outcome_id=4 OR Duel.outcome_id=14 OR Duel.outcome_id=15 OR Duel.outcome_id=16 OR Duel.outcome_id=17) AND Player.player_name=%s;"
        df_duels_won = pd.read_sql(query1,mydb, params=[player_name])
        query2 = "SELECT player_name, Duel.type_name FROM Event INNER JOIN Duel ON Event.id=Duel.id INNER JOIN Player ON Event.player_id=Player.player_id WHERE Player.player_name=%s;"
        df_duels_tot = pd.read_sql(query2,mydb, params=[player_name])
        mydb.close() #close the connection
        if len(df_duels_tot)==0:
            return 0
        else:
            perc = (len(df_duels_won)/len(df_duels_tot))*100
            return perc
        
    except Exception as e:
        mydb.close()
        print(str(e))

## Blocks p90

In [ ]:
def getBlocksPer90(player_name, team_name):
    try:
        player_min = getMinutesPlayed(player_name, team_name)/90
        mydb = connection.connect(host="localhost", database='mydb',user="root", passwd="",use_pure=True)
        query = "SELECT player_name, Block.type_name FROM Event INNER JOIN Block ON Event.id=Block.id INNER JOIN Player ON Event.player_id=Player.player_id WHERE Player.player_name=%s;"
        df_blocks = pd.read_sql(query,mydb, params=[player_name])
        mydb.close() #close the connection
        if player_min==0:
            return 0
        else:
            return len(df_blocks)/player_min
        
    except Exception as e:
        mydb.close()
        print(str(e))

## Interceptions Percentage

In [ ]:
def getInterceptionsPercentage(player_name):
    try:
        mydb = connection.connect(host="localhost", database='mydb',user="root", passwd="",use_pure=True)
        query1 = "SELECT player_name, Interception.type_name FROM Event INNER JOIN Interception ON Event.id=Interception.id INNER JOIN Player ON Event.player_id=Player.player_id WHERE (Interception.outcome_id=4 OR Interception.outcome_id=14 OR Interception.outcome_id=15 OR Interception.outcome_id=16 OR Interception.outcome_id=17) AND Player.player_name=%s;"
        df_interc_won = pd.read_sql(query1,mydb, params=[player_name])
        query2 = "SELECT player_name, Interception.type_name FROM Event INNER JOIN Interception ON Event.id=Interception.id INNER JOIN Player ON Event.player_id=Player.player_id WHERE Player.player_name=%s;"
        df_interc_tot = pd.read_sql(query2,mydb, params=[player_name])
        mydb.close() #close the connection
        if (len(df_interc_tot))==0:
            return 0
        else:
            perc = (len(df_interc_won)/len(df_interc_tot))*100
            return perc
       
    except Exception as e:
        mydb.close()
        print(str(e))

## Key Passes p90

In [ ]:
def getKeyPassesPer90(player_name, team_name):
    try:
        player_min = getMinutesPlayed(player_name, team_name)/90
        mydb = connection.connect(host="localhost", database='mydb',user="root", passwd="",use_pure=True)
        query = "SELECT player_name, Pass.type_name FROM Event INNER JOIN Pass ON Event.id=Pass.id INNER JOIN Player ON Event.player_id=Player.player_id INNER JOIN Shot ON Pass.id=Shot.shot_key_pass_id WHERE Player.player_name=%s;"
        df_keyp = pd.read_sql(query,mydb, params=[player_name])
        mydb.close() #close the connection
        if player_min==0:
            return 0
        else:
            return len(df_keyp)/player_min
        
    except Exception as e:
        mydb.close()
        print(str(e))

## xG Assisted p90

In [ ]:
def getxGAssistedPer90(player_name, team_name):
    try:
        player_min = getMinutesPlayed(player_name, team_name)/90
        mydb = connection.connect(host="localhost", database='mydb',user="root", passwd="",use_pure=True)
        query = "SELECT player_name, Pass.type_name, Shot.shot_statsbomb_xg FROM Event INNER JOIN Pass ON Event.id=Pass.id INNER JOIN Player ON Event.player_id=Player.player_id INNER JOIN Shot ON Pass.pass_assisted_shot_id=Shot.id WHERE Player.player_name=%s;"
        df_xGAss = pd.read_sql(query,mydb, params=[player_name])
        mydb.close() #close the connection
        if player_min==0:
            return 0
        else:
            return (df_xGAss['shot_statsbomb_xg'].sum())/player_min
        
    except Exception as e:
        mydb.close()
        print(str(e))

## Save Rate

In [ ]:
def getSaveRate(player_name):
    try:
        mydb = connection.connect(host="localhost", database='mydb',user="root", passwd="",use_pure=True)
        query1 = "SELECT match_id, sub_type_name FROM Event INNER JOIN GoalKeeper ON Event.id=GoalKeeper.id INNER JOIN Player ON Event.player_id=Player.player_id INNER JOIN SubType ON GoalKeeper.sub_type_id=SubType.sub_type_id WHERE (Event.period=1 OR Event.period=2 OR Event.period=3 OR Event.period=4) AND (GoalKeeper.sub_type_id=29 OR GoalKeeper.sub_type_id=33 OR GoalKeeper.sub_type_id=109 OR GoalKeeper.sub_type_id=114 ) AND Player.player_name=%s;"
        df_saves = pd.read_sql(query1,mydb, params=[player_name])
        query2 = "SELECT match_id, sub_type_name FROM Event INNER JOIN GoalKeeper ON Event.id=GoalKeeper.id INNER JOIN Player ON Event.player_id=Player.player_id INNER JOIN SubType ON GoalKeeper.sub_type_id=SubType.sub_type_id WHERE (Event.period=1 OR Event.period=2 OR Event.period=3 OR Event.period=4) AND (GoalKeeper.sub_type_id=29 OR GoalKeeper.sub_type_id=33 OR GoalKeeper.sub_type_id=109 OR GoalKeeper.sub_type_id=114 OR GoalKeeper.sub_type_id=26 OR GoalKeeper.sub_type_id=28) AND Player.player_name=%s;"
        df_shotsag = pd.read_sql(query2,mydb, params=[player_name])
        mydb.close() #close the connection
        if (len(df_shotsag))==0:
            return 0
        else:
            perc = (len(df_saves)/len(df_shotsag))*100
            return perc
        
    except Exception as e:
        mydb.close()
        print(str(e))

## Shots Against p90

In [ ]:
def getShotsAgainstPer90(player_name, team_name):
    try:
        player_min = getMinutesPlayed(player_name, team_name)/90
        mydb = connection.connect(host="localhost", database='mydb',user="root", passwd="",use_pure=True)
        query = "SELECT match_id, sub_type_name FROM Event INNER JOIN GoalKeeper ON Event.id=GoalKeeper.id INNER JOIN Player ON Event.player_id=Player.player_id INNER JOIN SubType ON GoalKeeper.sub_type_id=SubType.sub_type_id WHERE (Event.period=1 OR Event.period=2 OR Event.period=3 OR Event.period=4) AND (GoalKeeper.sub_type_id=29 OR GoalKeeper.sub_type_id=33 OR GoalKeeper.sub_type_id=109 OR GoalKeeper.sub_type_id=114 OR GoalKeeper.sub_type_id=26 OR GoalKeeper.sub_type_id=28) AND Player.player_name=%s;"
        df_shots_against = pd.read_sql(query,mydb, params=[player_name])
        mydb.close() #close the connection
        if player_min==0:
            return 0
        else:
            return len(df_shots_against)/player_min
        
    except Exception as e:
        mydb.close()
        print(str(e))

## xG Against p90

In [ ]:
def getxGAgainstPer90(player_name, team_name):
    try:
        player_min = getMinutesPlayed(player_name, team_name)/90
        mydb = connection.connect(host="localhost", database='mydb',user="root", passwd="",use_pure=True)
        query = "SELECT match_id, sub_type_name, Shot.shot_statsbomb_xg FROM Event INNER JOIN EventRelated ON Event.id=EventRelated.id_related INNER JOIN Shot on EventRelated.id=Shot.id INNER JOIN GoalKeeper ON Event.id=GoalKeeper.id INNER JOIN Player ON Event.player_id=Player.player_id INNER JOIN SubType ON GoalKeeper.sub_type_id=SubType.sub_type_id WHERE (Event.period=1 OR Event.period=2 OR Event.period=3 OR Event.period=4) AND (GoalKeeper.sub_type_id=29 OR GoalKeeper.sub_type_id=33 OR GoalKeeper.sub_type_id=109 OR GoalKeeper.sub_type_id=114 OR GoalKeeper.sub_type_id=26 OR GoalKeeper.sub_type_id=28) AND Player.player_name=%s;"
        df_xG_against = pd.read_sql(query,mydb, params=[player_name])
        mydb.close() #close the connection
        if player_min==0:
            return 0
        else:
            return (df_xG_against['shot_statsbomb_xg'].sum())/player_min
        
    except Exception as e:
        mydb.close()
        print(str(e))

## Keeper Sweeper Actions and Distance p90

In [ ]:
def getKeeperSweeperActionsPer90(player_name, team_name):
    try:
        player_min = getMinutesPlayed(player_name, team_name)/90
        mydb = connection.connect(host="localhost", database='mydb',user="root", passwd="",use_pure=True)
        query = "SELECT match_id, Event.x FROM Event INNER JOIN GoalKeeper ON Event.id=GoalKeeper.id INNER JOIN Player ON Event.player_id=Player.player_id WHERE GoalKeeper.sub_type_id=27 AND Player.player_name=%s;"
        df_ksa = pd.read_sql(query,mydb, params=[player_name])
        mydb.close() #close the connection
        if player_min==0:
            return 0
        else:
            return len(df_ksa)/player_min
        
    except Exception as e:
        mydb.close()
        print(str(e))

In [ ]:
def getKsaDistance(player_name):
    try:
        mydb = connection.connect(host="localhost", database='mydb',user="root", passwd="",use_pure=True)
        query = "SELECT match_id, Event.x FROM Event INNER JOIN GoalKeeper ON Event.id=GoalKeeper.id INNER JOIN Player ON Event.player_id=Player.player_id WHERE GoalKeeper.sub_type_id=27 AND Player.player_name=%s;"
        df_ksa = pd.read_sql(query,mydb, params=[player_name])
        mydb.close() #close the connection
        if len(df_ksa)==0:
            return 0
        else:
            return (df_ksa['x'].sum())/len(df_ksa)
        
    except Exception as e:
        mydb.close()
        print(str(e))

## Passing Percentage Under Pressure

In [ ]:
def getPassingPercentageUnderPressure(player_name):
    try:
        mydb = connection.connect(host="localhost", database='mydb',user="root", passwd="",use_pure=True)
        query1 = "SELECT player_name, Pass.type_name FROM Event INNER JOIN Pass ON Event.id=Pass.id INNER JOIN Player ON Event.player_id=Player.player_id WHERE Pass.outcome_id IS NULL AND Event.under_pressure=1 AND Player.player_name=%s;"
        df_passes_compl = pd.read_sql(query1,mydb, params=[player_name])
        query2 = "SELECT player_name, Pass.type_name FROM Event INNER JOIN Pass ON Event.id=Pass.id INNER JOIN Player ON Event.player_id=Player.player_id WHERE Pass.outcome_id IS NOT NULL AND Event.under_pressure=1 AND Player.player_name=%s;"
        df_passes_notcompl = pd.read_sql(query2,mydb, params=[player_name])
        mydb.close() #close the connection
        if (len(df_passes_compl)+len(df_passes_notcompl))==0:
            return 0
        else:
            perc = len(df_passes_compl)/(len(df_passes_compl)+len(df_passes_notcompl))*100
            return perc
        
    except Exception as e:
        mydb.close()
        print(str(e))

## Goals

In [ ]:
def getNumberOfGoals(player_name):
    try:
        mydb = connection.connect(host="localhost", database='mydb',user="root", passwd="",use_pure=True)
        query = "SELECT player_name, match_id FROM Event INNER JOIN Shot ON Event.id=Shot.id INNER JOIN Player ON Event.player_id=Player.player_id WHERE (Event.period=1 OR Event.period=2 OR Event.period=3 OR Event.period=4) AND Shot.outcome_id=97 AND Player.player_name=%s;"
        df_goals = pd.read_sql(query,mydb, params=[player_name])
        mydb.close() #close the connection
        return len(df_goals)
        
    except Exception as e:
        mydb.close()
        print(str(e))

In [ ]:
def getNumberOfGoalsPer90(player_name, team_name):
    try:
        player_min = getMinutesPlayed(player_name, team_name)/90
        mydb = connection.connect(host="localhost", database='mydb',user="root", passwd="",use_pure=True)
        query = "SELECT player_name, match_id FROM Event INNER JOIN Shot ON Event.id=Shot.id INNER JOIN Player ON Event.player_id=Player.player_id WHERE (Event.period=1 OR Event.period=2 OR Event.period=3 OR Event.period=4) AND Shot.outcome_id=97 AND Player.player_name=%s;"
        df_goals = pd.read_sql(query,mydb, params=[player_name])
        mydb.close() #close the connection
        if player_min==0:
            return 0
        else:
            return len(df_goals)/player_min
        
    except Exception as e:
        mydb.close()
        print(str(e))

## Assists

In [ ]:
def getNumberOfAssists(player_name):
    try:
        mydb = connection.connect(host="localhost", database='mydb',user="root", passwd="",use_pure=True)
        query = "SELECT player_name, match_id FROM Event INNER JOIN Pass ON Event.id=Pass.id INNER JOIN Player ON Event.player_id=Player.player_id WHERE Pass.pass_goal_assist=1 AND Player.player_name=%s;"
        df_assists = pd.read_sql(query,mydb, params=[player_name])
        mydb.close() #close the connection
        return len(df_assists)
        
    except Exception as e:
        mydb.close()
        print(str(e))

## Yellow Card

In [ ]:
def getNumberOfYellowCards(player_name):
    try:
        mydb = connection.connect(host="localhost", database='mydb',user="root", passwd="",use_pure=True)
        query_bb = "SELECT player_name, match_id FROM Event INNER JOIN BadBehaviour ON Event.id=BadBehaviour.id INNER JOIN Player ON Event.player_id=Player.player_id WHERE BadBehaviour.bad_behaviour_card_name='Yellow Card' AND Player.player_name=%s;"
        df_bb_yc = pd.read_sql(query_bb,mydb, params=[player_name])
        query_fc = "SELECT player_name, match_id FROM Event INNER JOIN FoulCommitted ON Event.id=FoulCommitted.id INNER JOIN Player ON Event.player_id=Player.player_id WHERE FoulCommitted.foul_committed_card_name='Yellow Card' AND Player.player_name=%s;"
        df_fc_yc = pd.read_sql(query_fc,mydb, params=[player_name])
        mydb.close() #close the connection
        return (len(df_bb_yc)+len(df_fc_yc))
        
    except Exception as e:
        mydb.close()
        print(str(e))

## Red Card

In [ ]:
def getNumberOfRedCards(player_name):
    try:
        mydb = connection.connect(host="localhost", database='mydb',user="root", passwd="",use_pure=True)
        query_bb = "SELECT player_name, match_id FROM Event INNER JOIN BadBehaviour ON Event.id=BadBehaviour.id INNER JOIN Player ON Event.player_id=Player.player_id WHERE (BadBehaviour.bad_behaviour_card_name='Second Yellow' OR BadBehaviour.bad_behaviour_card_name='Red Card') AND Player.player_name=%s;"
        df_bb_rc = pd.read_sql(query_bb,mydb, params=[player_name])
        query_fc = "SELECT player_name, match_id FROM Event INNER JOIN FoulCommitted ON Event.id=FoulCommitted.id INNER JOIN Player ON Event.player_id=Player.player_id WHERE (FoulCommitted.foul_committed_card_name=' Second Yellow' OR FoulCommitted.foul_committed_card_name='Red Card') AND Player.player_name=%s;"
        df_fc_rc = pd.read_sql(query_fc,mydb, params=[player_name])
        mydb.close() #close the connection
        return (len(df_bb_rc)+len(df_fc_rc))
        
    except Exception as e:
        mydb.close()
        print(str(e))

## Goals Conceded

In [ ]:
def getGoalsConceded(player_name):
    try:
        mydb = connection.connect(host="localhost", database='mydb',user="root", passwd="",use_pure=True)
        query = "SELECT player_name, match_id FROM Event INNER JOIN GoalKeeper ON Event.id=GoalKeeper.id INNER JOIN Player ON Event.player_id=Player.player_id WHERE (Event.period=1 OR Event.period=2 OR Event.period=3 OR Event.period=4) AND (GoalKeeper.sub_type_id=26 OR GoalKeeper.sub_type_id=28) AND Player.player_name=%s;"
        df_goals_conc = pd.read_sql(query,mydb, params=[player_name])
        mydb.close() #close the connection
        return len(df_goals_conc)
        
    except Exception as e:
        mydb.close()
        print(str(e))

In [ ]:
def getGoalsConcededPer90(player_name, team_name):
    try:
        player_min = getMinutesPlayed(player_name, team_name)/90
        mydb = connection.connect(host="localhost", database='mydb',user="root", passwd="",use_pure=True)
        query = "SELECT player_name, match_id FROM Event INNER JOIN GoalKeeper ON Event.id=GoalKeeper.id INNER JOIN Player ON Event.player_id=Player.player_id WHERE (Event.period=1 OR Event.period=2 OR Event.period=3 OR Event.period=4) AND (GoalKeeper.sub_type_id=26 OR GoalKeeper.sub_type_id=28) AND Player.player_name=%s;"
        df_goals_conc = pd.read_sql(query,mydb, params=[player_name])
        mydb.close() #close the connection
        if player_min==0:
            return 0
        else:
            return len(df_goals_conc)/player_min
        
    except Exception as e:
        mydb.close()
        print(str(e))

## Presences

In [ ]:
def getNumberOfPresences(player_name, team_name):
    dict_id_lambda = list(map(lambda x: x['match_id'], match_dict_home[team_name]))
    player_pres_tot = 0
    for i in dict_id_lambda:
        try:
            mydb = connection.connect(host="localhost", database='mydb',user="root", passwd="",use_pure=True)
            query = "SELECT match_id, Player.player_name FROM mydb.Event INNER JOIN Player ON Event.player_id=Player.Player_id WHERE Event.position_id!=26 AND Event.match_id=%s AND Player.player_name=%s;"
            df_event = pd.read_sql(query,mydb, params=[i, player_name])
            mydb.close() #close the connection
        except Exception as e:
            mydb.close()
            print(str(e))
        if df_event['player_name'].str.contains(player_name).any() == True:
            player_pres_tot = player_pres_tot + 1
            
    return player_pres_tot

## Aerial Win Percentage

In [ ]:
def getAerialsWinPercentage(player_name):
    try:
        mydb = connection.connect(host="localhost", database='mydb',user="root", passwd="",use_pure=True)
        query1 = "SELECT player_name, match_id FROM Event INNER JOIN Player ON Event.player_id=Player.player_id WHERE aerial_won=1 AND Player.player_name=%s;"
        df_aerial_won = pd.read_sql(query1,mydb, params=[player_name])
        query2 = "SELECT player_name, match_id FROM Event INNER JOIN Duel ON Event.id=Duel.id INNER JOIN Player ON Event.player_id=Player.player_id WHERE Duel.sub_type_id=10 AND Player.player_name=%s;"
        df_aerial_notwon = pd.read_sql(query2,mydb, params=[player_name])
        mydb.close() #close the connection
        if (len(df_aerial_won)+len(df_aerial_notwon))==0:
            return 0
        else:
            perc = len(df_aerial_won)/(len(df_aerial_won)+len(df_aerial_notwon))*100
            return perc
        
    except Exception as e:
        mydb.close()
        print(str(e))

## Tackles and Interceptions p90

In [ ]:
def getTacklesAndInterceptionsPer90(player_name, team_name):
    try:
        player_min = getMinutesPlayed(player_name, team_name)/90
        mydb = connection.connect(host="localhost", database='mydb',user="root", passwd="",use_pure=True)
        query1 = "SELECT player_name, match_id FROM Event INNER JOIN Interception ON Event.id=Interception.id INNER JOIN Player ON Event.player_id=Player.player_id WHERE Player.player_name=%s;"
        df_interc = pd.read_sql(query1,mydb, params=[player_name])
        query2 = "SELECT player_name, match_id FROM Event INNER JOIN Duel ON Event.id=Duel.id INNER JOIN Player ON Event.player_id=Player.player_id WHERE Duel.sub_type_id=11 AND Player.player_name=%s;"
        df_tackles = pd.read_sql(query2,mydb, params=[player_name])
        mydb.close() #close the connection
        interc_tackle_tot = len(df_interc) + len(df_tackles)
        if player_min==0:
            return 0
        else:
            return interc_tackle_tot/player_min
        
    except Exception as e:
        mydb.close()
        print(str(e))

## Tackle and Interception Percentage

In [ ]:
def getTackleInterceptionPercentage(player_name):
    try:
        mydb = connection.connect(host="localhost", database='mydb',user="root", passwd="",use_pure=True)
        query1 = "SELECT player_name, match_id, Interception.outcome_id FROM Event INNER JOIN Interception ON Event.id=Interception.id INNER JOIN Player ON Event.player_id=Player.player_id WHERE (Interception.outcome_id=4 OR Interception.outcome_id=14 OR Interception.outcome_id=15 OR Interception.outcome_id=16 OR Interception.outcome_id=17) AND Player.player_name=%s;"
        df_interc_won = pd.read_sql(query1,mydb, params=[player_name])
        query2 = "SELECT player_name, match_id FROM Event INNER JOIN Duel ON Event.id=Duel.id INNER JOIN Player ON Event.player_id=Player.player_id WHERE Duel.sub_type_id=11 AND (Duel.outcome_id=4 OR Duel.outcome_id=14 OR Duel.outcome_id=15 OR Duel.outcome_id=16 OR Duel.outcome_id=17) AND Player.player_name=%s;"
        df_tackles_won = pd.read_sql(query2,mydb, params=[player_name])
        query3 = "SELECT player_name, match_id, Interception.outcome_id FROM Event INNER JOIN Interception ON Event.id=Interception.id INNER JOIN Player ON Event.player_id=Player.player_id WHERE Player.player_name=%s;"
        df_interc_tot = pd.read_sql(query3,mydb, params=[player_name])
        query4 = "SELECT player_name, match_id FROM Event INNER JOIN Duel ON Event.id=Duel.id INNER JOIN Player ON Event.player_id=Player.player_id WHERE Duel.sub_type_id=11 AND Player.player_name=%s;"
        df_tackles_tot = pd.read_sql(query4,mydb, params=[player_name])
        mydb.close() #close the connection
        if (len(df_interc_tot) + len(df_tackles_tot))==0:
            return 0
        else:
            perc = (len(df_interc_won) + len(df_tackles_won))/(len(df_interc_tot) + len(df_tackles_tot))*100
            return perc
        
    except Exception as e:
        mydb.close()
        print(str(e))

## Player Heatmap

In [ ]:
def generatePlayerHeatMap(player_name, match_name, match_name_dict):
    match_id = match_name_dict[match_name]
    try:
        mydb = connection.connect(host="localhost", database='mydb',user="root", passwd="",use_pure=True)
        query = "SELECT x, y, end_x, end_y FROM mydb.Event INNER JOIN Pass ON  Event.id=Pass.id INNER JOIN Player ON Event.player_id=Player.Player_id WHERE Event.match_id=%s AND Player.player_name=%s;"
        player_df = pd.read_sql(query,mydb, params=[match_id, player_name])
        mydb.close() #close the connection
    except Exception as e:
        mydb.close()
        print(str(e))

    pitch = Pitch(line_color='white',pitch_color='#02540b')
    fig, ax = pitch.grid(figheight=6, grid_height=0.9, title_height=0.02, axis=False,endnote_height=0, title_space=0, endnote_space=0)
    #Create the heatmap
    pitch.kdeplot(
        x=player_df['x'],
        y=player_df['y'],
        shade = True,
        shade_lowest=False,
        alpha=.5,
        n_levels=10,
        cmap = 'plasma',
        ax=ax['pitch']
        )
    fig.suptitle(player_name+" Heatmap", fontsize = 14)
    
    return fig

## Radar

In [ ]:
player_roles_dict = {'Goalkeeper': 'Goalkeeper',
                     'Right Back': 'Full-back/Wing-back',
                     'Left Back': 'Full-back/Wing-back', 
                     'Right Wing Back': 'Full-back/Wing-back', 
                     'Left Wing Back': 'Full-back/Wing-back',
                     'Center Back': 'Center-back', 
                     'Right Center Back': 'Center-back', 
                     'Left Center Back': 'Center-back',
                     'Right Defensive Midfield': 'Midfielder',
                     'Center Defensive Midfield': 'Midfielder',
                     'Left Defensive Midfield': 'Midfielder',
                     'Right Midfield': 'Midfielder', 
                     'Right Center Midfield': 'Midfielder',
                     'Center Midfield': 'Midfielder', 
                     'Left Center Midfield': 'Midfielder',
                     'Right Midfield': 'Wide Midfielder/Attacking Midfielder',
                     'Left Midfield': 'Wide Midfielder/Attacking Midfielder', 
                     'Right Wing': 'Wide Midfielder/Attacking Midfielder',
                     'Left Wing': 'Wide Midfielder/Attacking Midfielder', 
                     'Right Attacking Midfield': 'Wide Midfielder/Attacking Midfielder', 
                     'Left Attacking Midfield': 'Wide Midfielder/Attacking Midfielder', 
                     'Center Attacking Midfield': 'Wide Midfielder/Attacking Midfielder',
                     'Right Center Forward': 'Striker', 
                     'Striker': 'Striker', 
                     'Center Forward': 'Striker',
                     'Left Center Forward': 'Striker', 
                     'Secondary Striker': 'Striker',
                     'No Events': 'No Events',
                     'Substitute': 'No Events'
                     }

In [ ]:
# parameter names of the statistics we want to show
gk_params = ["Passing %", "KS Actions p90", "KSA Avg. Distance", "xG Against p90", "Goals Against p90"]
center_back_params = ["Passing %", "Pressures p90", "Fouls p90", "Carries p90", "Duels Win %", "Interceptions Win %", "Blocks p90", "Clearances p90", "Aerials Win %"]
full_back_wing_back_params = ["Passing %", "Pressures p90", "Fouls p90", "Duels Win %", "Interceptions Win %", "Dispossessions p90", "Dribbles Completed p90", "Crossing %", "Aerials Win %"]
midfielder_params = ["Passing %", "Dribbles Completed p90", "Fouls Won p90", "Dispossessions p90", "Pressures p90", "Duels Win %", "Interceptions Win %", "Key Passes p90", "xG p90", "xG Assisted p90"]
wide_attacking_midfielder_params = ["xG p90", "Shots p90", "xG/Shot", "Passing %", "Fouls Won p90", "Dribbles Completed p90", "Dispossessions p90", "Pressures p90", "xG Assisted p90", "Key Passes p90"]
striker_params = ["xG p90", "Shots p90", "xG/Shot", "xG Assisted p90", "Dribbles Completed p90", "Key Passes p90", "Pressures p90", "Dispossessions p90", "Passing %", "Aerials Win %"]

# The lower and upper boundaries for the statistics
low_gk = [60, 0.19, 5, 0.3, 0.41]
high_gk = [84, 1.37, 19, 1.08, 2.39]

# The lower and upper boundaries for the statistics
low_center_back =  [71, 0, 0, 18, 7, 24, 0.5, 1.6, 34]
high_center_back = [98, 13, 2.5, 74, 63, 96, 2.9, 7.3, 88]

# The lower and upper boundaries for the statistics
low_full_wing_back =  [69, 6, 0, 20, 22, 0, 0, 0, 16]
high_full_wing_back = [87, 20, 3.1, 78, 100, 1.8, 3.1, 47, 82]

# The lower and upper boundaries for the statistics
low_midfielder = [76, 0, 0, 0, 1, 16, 13, 0, 0, 0]
high_midfielder = [93, 1.8, 3.1, 3.5, 31, 71, 97, 2.5, 0.20, 0.16]

# The lower and upper boundaries for the statistics
low_wide_attacking_midf =  [0, 0, 0, 59, 0, 0, 0.15, 0, 0, 0]
high_wide_attacking_midf = [0.62, 7, 0.17, 87, 4, 4.5, 3.75, 43, 1, 7.4]

# The lower and upper boundaries for the statistics
low_striker =  [0, 0, 0, 0, 0, 0, 0, 0, 51, 12]
high_striker = [1.13, 8, 0.20, 0.31, 1.95, 2.2, 55, 3.4, 85, 60]

# Add anything to this list where having a lower number is better
# this flips the statistic
lower_is_better = ["Goals Against p90", "Dispossessions p90", "Fouls p90"]



URL1 = ('https://raw.githubusercontent.com/googlefonts/SourceSerifProGFVersion/main/fonts/'
        'SourceSerifPro-Regular.ttf')
serif_regular = FontManager(URL1)
URL2 = ('https://raw.githubusercontent.com/googlefonts/SourceSerifProGFVersion/main/fonts/'
        'SourceSerifPro-ExtraLight.ttf')
serif_extra_light = FontManager(URL2)
URL3 = ('https://raw.githubusercontent.com/google/fonts/main/ofl/rubikmonoone/'
        'RubikMonoOne-Regular.ttf')
rubik_regular = FontManager(URL3)
URL4 = 'https://raw.githubusercontent.com/googlefonts/roboto/main/src/hinted/Roboto-Thin.ttf'
robotto_thin = FontManager(URL4)
URL5 = ('https://raw.githubusercontent.com/google/fonts/main/apache/robotoslab/'
        'RobotoSlab%5Bwght%5D.ttf')
robotto_bold = FontManager(URL5)

def generateRadarFig(player_values, player_role, player_name, team_name, params, low_params, high_params):
    
    
    radar = Radar(params, low_params, high_params,
              lower_is_better=lower_is_better,
              # whether to round any of the labels to integers instead of decimal places
              round_int=[False]*len(params),
              num_rings=4,  # the number of concentric circles (excluding center circle)
              # if the ring_width is more than the center_circle_radius then
              # the center circle radius will be wider than the width of the concentric circles
              ring_width=1, center_circle_radius=1)
    
    
    # creating the figure using the grid function from mplsoccer:
    fig, axs = grid(figheight=10, grid_height=0.915, title_height=0.06, endnote_height=0.025,
                    title_space=0, endnote_space=0, grid_key='radar', axis=False)
    # plot the radar
    radar.setup_axis(ax=axs['radar'])
    rings_inner = radar.draw_circles(ax=axs['radar'], facecolor='#ffb2b2', edgecolor='#fc5f5f')
    
        
    radar_output = radar.draw_radar(player_values, ax=axs['radar'],
                                        kwargs_radar={'facecolor': '#aa65b2'},
                                        kwargs_rings={'facecolor': '#66d8ba'})

        
    

    range_labels = radar.draw_range_labels(ax=axs['radar'], fontsize=25,
                                           fontproperties=robotto_thin.prop)
    param_labels = radar.draw_param_labels(ax=axs['radar'], fontsize=25,
                                           fontproperties=robotto_thin.prop)

    # adding the endnote and title text (these axes range from 0-1, i.e. 0, 0 is the bottom left)
    # Note we are slightly offsetting the text from the edges by 0.01 (1%, e.g. 0.99)
    endnote_text = axs['endnote'].text(0.99, 0.5, 'Inspired By: StatsBomb / Rami Moghadam', fontsize=15,
                                       fontproperties=robotto_thin.prop, ha='right', va='center')
    title1_text = axs['title'].text(0.01, 0.65, player_name, fontsize=25,
                                    fontproperties=robotto_bold.prop, ha='left', va='center')
    title2_text = axs['title'].text(0.01, 0.25, team_name, fontsize=20,
                                    fontproperties=robotto_thin.prop,
                                    ha='left', va='center', color='#B6282F')
    title3_text = axs['title'].text(0.99, 0.65, 'Radar Chart', fontsize=25,
                                    fontproperties=robotto_bold.prop, ha='right', va='center')
    title4_text = axs['title'].text(0.99, 0.25, player_role, fontsize=20,
                                    fontproperties=robotto_thin.prop,
                                    ha='right', va='center', color='#B6282F')
    return fig


def generatePlayerRadar(player_name1, team_name1, player_name2, team_name2):
    
    player_values2 = None
    player_role, player_values, params, low_params, high_params = calculateRadarPlayerValues(player_name1, team_name1)
    if (player_name2!='Select a player'):
        _, player_values2, _, _, _ = calculateRadarPlayerValues(player_name2, team_name2)
        return generateCompareRadarFig(player_values, player_values2, player_role, player_name1, player_name2, team_name1, team_name2, params, low_params, high_params)
    
    else:
        return generateRadarFig(player_values, player_role, player_name1, team_name1, params, low_params, high_params)
        
    
    
    
def calculateRadarPlayerValues(player_name, team_name):

    player_role = player_roles_dict[findPlayerRole(player_name)]
    
    if player_role == 'Goalkeeper':
        player_values = [getPassingPercentage(player_name), getKeeperSweeperActionsPer90(player_name, team_name), getKsaDistance(player_name), getxGAgainstPer90(player_name, team_name), getGoalsConcededPer90(player_name, team_name)]
        return player_role, player_values, gk_params, low_gk, high_gk
    
    if player_role == 'Center-back':
        player_values = [getPassingPercentage(player_name), getPressuresPer90(player_name, team_name), getFoulsCommittedPer90(player_name, team_name), getCarriesPer90(player_name, team_name), getDuelsWonPercentage(player_name), getInterceptionsPercentage(player_name), getBlocksPer90(player_name, team_name), getClearancesPer90(player_name, team_name), getAerialsWinPercentage(player_name)]
        return player_role, player_values, center_back_params, low_center_back, high_center_back
        
    if player_role == 'Full-back/Wing-back':
        player_values = [getPassingPercentage(player_name), getPressuresPer90(player_name, team_name), getFoulsCommittedPer90(player_name, team_name), getDuelsWonPercentage(player_name), getInterceptionsPercentage(player_name), DispossessionsPer90(player_name, team_name), getDribblesCompletedPer90(player_name, team_name), getCrossesPercentage(player_name), getAerialsWinPercentage(player_name)]
        return player_role, player_values, full_back_wing_back_params, low_full_wing_back, high_full_wing_back
    
    if player_role == 'Midfielder':
        player_values = [getPassingPercentage(player_name), getDribblesCompletedPer90(player_name, team_name), getFoulsWonPer90(player_name, team_name), getDispossessionsPer90(player_name, team_name), getPressuresPer90(player_name, team_name), getDuelsWonPercentage(player_name), getInterceptionsPercentage(player_name), getKeyPassesPer90(player_name, team_name), getxGValuePer90(player_name, team_name), getxGAssistedPer90(player_name, team_name)]
        return player_role, player_values, midfielder_params, low_midfielder, high_midfielder
        
        
    if player_role == 'Wide Midfielder/Attacking Midfielder':
        player_values = [getxGValuePer90(player_name, team_name), getShotsValuePer90(player_name, team_name), getxGperShotValue(player_name, team_name), getPassingPercentage(player_name), getFoulsWonPer90(player_name, team_name), getDribblesCompletedPer90(player_name, team_name), getDispossessionsPer90(player_name, team_name), getPressuresPer90(player_name, team_name), getxGAssistedPer90(player_name, team_name), getKeyPassesPer90(player_name, team_name)]
        return player_role, player_values, wide_attacking_midfielder_params, low_wide_attacking_midf, high_wide_attacking_midf
    
    if player_role == 'Striker':
        player_values = [getxGValuePer90(player_name, team_name), getShotsValuePer90(player_name, team_name), getxGperShotValue(player_name, team_name), getxGAssistedPer90(player_name, team_name), getDribblesCompletedPer90(player_name, team_name), getKeyPassesPer90(player_name, team_name), getPressuresPer90(player_name, team_name), getDispossessionsPer90(player_name, team_name), getPassingPercentage(player_name), getAerialsWinPercentage(player_name)]
        return player_role, player_values, striker_params, low_striker, high_striker

    
    

In [ ]:
def generateCompareRadarFig(player_values, player_values2, player_role, player_name, player_name2, team_name, team_name2, params, low_params, high_params):
    radar = Radar(params, low_params, high_params,
              lower_is_better=lower_is_better,
              # whether to round any of the labels to integers instead of decimal places
              round_int=[False]*len(params),
              num_rings=4,  # the number of concentric circles (excluding center circle)
              # if the ring_width is more than the center_circle_radius then
              # the center circle radius will be wider than the width of the concentric circles
              ring_width=1, center_circle_radius=1)
    # creating the figure using the grid function from mplsoccer:
    fig, axs = grid(figheight=10, grid_height=0.915, title_height=0.06, endnote_height=0.025,
                    title_space=0, endnote_space=0, grid_key='radar', axis=False)

    # plot radar
    radar.setup_axis(ax=axs['radar'])  # format axis as a radar
    rings_inner = radar.draw_circles(ax=axs['radar'], facecolor='#ffb2b2', edgecolor='#fc5f5f')
    radar_output = radar.draw_radar_compare(player_values, player_values2, ax=axs['radar'],
                                            kwargs_radar={'facecolor': '#00f2c1', 'alpha': 0.6},
                                            kwargs_compare={'facecolor': '#d80499', 'alpha': 0.6})
    radar_poly, radar_poly2, vertices1, vertices2 = radar_output
    range_labels = radar.draw_range_labels(ax=axs['radar'], fontsize=25,
                                           fontproperties=robotto_thin.prop)
    param_labels = radar.draw_param_labels(ax=axs['radar'], fontsize=25,
                                           fontproperties=robotto_thin.prop)
    axs['radar'].scatter(vertices1[:, 0], vertices1[:, 1],
                         c='#00f2c1', edgecolors='#6d6c6d', marker='o', s=150, zorder=2)
    axs['radar'].scatter(vertices2[:, 0], vertices2[:, 1],
                         c='#d80499', edgecolors='#6d6c6d', marker='o', s=150, zorder=2)

    # adding the endnote and title text (these axes range from 0-1, i.e. 0, 0 is the bottom left)
    # Note we are slightly offsetting the text from the edges by 0.01 (1%, e.g. 0.99)
    endnote_text = axs['endnote'].text(0.99, 0.5, 'Inspired By: StatsBomb / Rami Moghadam', fontsize=15,
                                       fontproperties=robotto_thin.prop, ha='right', va='center')
    title1_text = axs['title'].text(0.01, 0.65, player_name, fontsize=25, color='#01c49d',
                                    fontproperties=robotto_bold.prop, ha='left', va='center')
    title2_text = axs['title'].text(0.01, 0.25, team_name, fontsize=20,
                                    fontproperties=robotto_thin.prop,
                                    ha='left', va='center', color='#01c49d')
    title3_text = axs['title'].text(0.99, 0.65, player_name2, fontsize=25,
                                    fontproperties=robotto_bold.prop,
                                    ha='right', va='center', color='#d80499')
    title4_text = axs['title'].text(0.99, 0.25, team_name2, fontsize=20,
                                    fontproperties=robotto_thin.prop,
                                    ha='right', va='center', color='#d80499')
    return fig

## Key Pass Map

In [ ]:
def getKeyPassPlot(player_name):
    #drawing pitch
    pitch = Pitch(line_color = "black")
    fig, ax = pitch.draw(figsize=(9, 6))
    try:
        mydb = connection.connect(host="localhost", database='mydb',user="root", passwd="",use_pure=True)
        query = "SELECT Event.id, x, y, end_x, end_y FROM Event INNER JOIN Pass ON Event.id=Pass.id INNER JOIN Player ON Event.player_id=Player.player_id INNER JOIN Shot ON Pass.id=Shot.shot_key_pass_id WHERE Player.player_name=%s;"
        player_key_passes = pd.read_sql(query,mydb, params=[player_name])
        mydb.close() #close the connection
    except Exception as e:
        mydb.close()
        print(str(e))
    for i, thepass in player_key_passes.iterrows():
        x=thepass['x']
        y=thepass['y']
        #plot circle
        passCircle=plt.Circle((x,y),2,color="blue")
        passCircle.set_alpha(.2)
        ax.add_patch(passCircle)
        dx=thepass['end_x']-x
        dy=thepass['end_y']-y
        #plot arrow
        passArrow=plt.Arrow(x,y,dx,dy,width=3,color="blue")
        ax.add_patch(passArrow)

    ax.set_title(player_name + " key-passes", fontsize = 24)
    plt.show()
    return fig    

## Cross Map

In [ ]:
def getCrossMap(player_name):
    #drawing pitch
    pitch = Pitch(line_color = "black")
    fig, ax = pitch.draw(figsize=(9, 6))
    try:
        mydb = connection.connect(host="localhost", database='mydb',user="root", passwd="",use_pure=True)
        query = "SELECT Event.id, x, y, end_x, end_y FROM Event INNER JOIN Pass ON Event.id=Pass.id INNER JOIN Player ON Event.player_id=Player.player_id WHERE pass_cross=1 AND Player.player_name=%s;"
        player_crosses = pd.read_sql(query,mydb, params=[player_name])
        mydb.close() #close the connection
    except Exception as e:
        mydb.close()
        print(str(e))
    for i, thecross in player_crosses.iterrows():
        x=thecross['x']
        y=thecross['y']
        #plot circle
        passCircle=plt.Circle((x,y),2,color="blue")
        passCircle.set_alpha(.2)
        ax.add_patch(passCircle)
        dx=thecross['end_x']-x
        dy=thecross['end_y']-y
        #plot arrow
        passArrow=plt.Arrow(x,y,dx,dy,width=3,color="blue")
        ax.add_patch(passArrow)

    ax.set_title(player_name + " crosses", fontsize = 24)
    plt.show()
    return fig

## Shot Map

In [ ]:
def getShotMap(player_name):
    #drawing pitch
    pitch = Pitch(line_color = "black")
    fig, ax = pitch.draw(figsize=(9, 6))
    # Plot the shot
    circleSize = 2
    try:
        mydb = connection.connect(host="localhost", database='mydb',user="root", passwd="",use_pure=True)
        query = "SELECT Event.id, x, y, end_x, end_y, SubType.sub_type_name, Outcome.outcome_name FROM Event INNER JOIN Shot ON Event.id=Shot.id INNER JOIN Player ON Event.player_id=Player.player_id INNER JOIN Outcome ON Shot.outcome_id=Outcome.outcome_id INNER JOIN SubType ON Shot.sub_type_id=SubType.sub_type_id WHERE SubType.sub_type_name!='Penalty' AND (period=1 OR period=2 OR period=3 OR period=4) AND Player.player_name=%s;"
        player_shots = pd.read_sql(query,mydb, params=[player_name])
        mydb.close() #close the connection
    except Exception as e:
        mydb.close()
        print(str(e))
    for i, theshot in player_shots.iterrows():
        x=theshot['x']
        y=theshot['y']

        if theshot['outcome_name'] == 'Goal' :
            color = "green"
        elif theshot['outcome_name'] == 'Saved':
            color = "lightcoral"
        elif theshot['outcome_name'] == 'Blocked':
            color = "orange"
        else:
            color = "gainsboro"
        #plot circle
        passCircle=plt.Circle((x,y),2,color=color)
        passCircle.set_alpha(.2)
        ax.add_patch(passCircle)
        dx=theshot['end_x']-x
        dy=theshot['end_y']-y
        #plot arrow
        passArrow=plt.Arrow(x,y,dx,dy,width=3,color=color)
        ax.add_patch(passArrow)
                
    green_patch = mpatches.Patch(color='green', label='Goal')
    lightcoral_patch = mpatches.Patch(color='lightcoral', label='Saved')
    orange_patch = mpatches.Patch(color='orange', label='Blocked')
    gainsboro_patch = mpatches.Patch(color='gainsboro', label='Out')
    plt.legend(handles=[green_patch, lightcoral_patch, orange_patch, gainsboro_patch])
    ax.set_title(player_name + " shots", fontsize = 24)
    fig.show() 
    return fig

## Def Actions Map

In [ ]:
def getDefActionMap(player_name):
    pitch = Pitch(line_color = "black")
    fig, ax = pitch.draw(figsize=(9, 6))
    # Plot the shot
    circleSize = 1
    interception_won = ['Won', 'Success', 'Success In Play', 'Success Out']
    
    try:
        mydb = connection.connect(host="localhost", database='mydb',user="root", passwd="",use_pure=True)
        query1 = "SELECT Event.id, x, y, Interception.type_name, Outcome.outcome_name FROM Event INNER JOIN Interception ON Event.id=Interception.id INNER JOIN Player ON Event.player_id=Player.player_id INNER JOIN Outcome ON Interception.outcome_id=Outcome.outcome_id WHERE Player.player_name=%s;"
        df_player_int = pd.read_sql(query1,mydb, params=[player_name])
        query2 = "SELECT Event.id, x, y, Block.type_name FROM Event INNER JOIN Block ON Event.id=Block.id INNER JOIN Player ON Event.player_id=Player.player_id WHERE Player.player_name=%s;"
        df_player_blocks = pd.read_sql(query2,mydb, params=[player_name])
        query3 = "SELECT Event.id, x, y, BallRecovery.type_name FROM Event INNER JOIN BallRecovery ON Event.id=BallRecovery.id INNER JOIN Player ON Event.player_id=Player.player_id WHERE Player.player_name=%s;"
        df_player_br = pd.read_sql(query3,mydb, params=[player_name])
        query4 = "SELECT Event.id, x, y, Clearance.type_name FROM Event INNER JOIN Clearance ON Event.id=Clearance.id INNER JOIN Player ON Event.player_id=Player.player_id WHERE Player.player_name=%s;"
        df_player_cl = pd.read_sql(query4,mydb, params=[player_name])
        def_actions = pd.concat([df_player_int, df_player_blocks, df_player_br, df_player_cl], ignore_index=True, sort=False)
        for i, theact in def_actions.iterrows():
            x=theact['x']
            y=theact['y']

            if theact['type_name'] == 'Interception' :
                color = "green"
                if theact['outcome_name'] in interception_won:
                    #plot circle
                    passCircle=plt.Circle((x,y),2,color=color)
                    passCircle.set_alpha(.2)
                    ax.add_patch(passCircle)
                else:
                    passCircle=plt.Circle((x,y),2,color=color, fill=False)
                    passCircle.set_alpha(.2)
                    ax.add_patch(passCircle)


            elif theact['type_name'] == 'Clearance':
                color = "red"
                passCircle=plt.Circle((x,y),2,color=color)
                passCircle.set_alpha(.2)
                ax.add_patch(passCircle),
            elif theact['type_name'] == 'Block':
                color = "orange"
                passCircle=plt.Circle((x,y),2,color=color)
                passCircle.set_alpha(.2)
                ax.add_patch(passCircle),
            elif theact['type_name'] == 'Ball Recovery':
                color = "black"
                passCircle=plt.Circle((x,y),2,color=color)
                passCircle.set_alpha(.2)
                ax.add_patch(passCircle)

        mydb.close() #close the connection
    except Exception as e:
        mydb.close()
        print(str(e))
                    
    green_patch = mpatches.Patch(color='green', label='Interception')
    red_patch = mpatches.Patch(color='red', label='Clearance')
    orange_patch = mpatches.Patch(color='orange', label='Block')
    black_patch = mpatches.Patch(color='black', label='Ball Recovery')
    plt.legend(handles=[green_patch, red_patch, orange_patch, black_patch])
    ax.set_title(player_name + " defensive actions", fontsize = 24)
    fig.show() 
    return fig

## GK Pass Map

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from mplsoccer import VerticalPitch

def getGkBuildUpPassesMap(player_name):    
    try:
        mydb = connection.connect(host="localhost", database='mydb',user="root", passwd="",use_pure=True)
        query = "SELECT Player.player_name, Team.team_name, x, y, end_x, end_y, Pass.pass_angle, Pass.pass_length FROM Event INNER JOIN Pass ON Event.id=Pass.id INNER JOIN Player ON Event.player_id=Player.player_id INNER JOIN Team ON Event.team_id=Team.team_id WHERE Player.player_name=%s;"
        pass_data = pd.read_sql(query,mydb, params=[player_name])
        mydb.close() #close the connection
    except Exception as e:
        mydb.close()
        print(str(e))    
        
    pass_data = pass_data[['player_name', 'team_name', 'x', 'y', 'end_x', 'end_y', 'pass_angle', 'pass_length']]
    pass_data_clean = pass_data[(pass_data['end_x'].isna() == False) & (pass_data['end_y'].isna() == False) & (pass_data['pass_length'].isna() == False)]
    X = pass_data_clean[['x', 'y', 'end_x', 'end_y', 'pass_angle', 'pass_length']]
    

    n = list(range(2, 11))
    errors = []

    pitch = VerticalPitch(pitch_type='statsbomb')
    fig, axs = pitch.grid(title_height=0.06, endnote_space=0.2, 
                          title_space=0, grid_height=0.50, 
                          endnote_height=0.05, axis=False)
    if len(X)==0:
        return fig
    
    for i in range(2, 11):
        model = KMeans(n_clusters=i, random_state=42)
        model.fit(X)
        errors.append(model.inertia_)

    model = KMeans(n_clusters=6, random_state=42)
    model.fit(X)

    pass_data_clean['cluster'] = model.labels_
    
    colors = ['red', 'orange', 'green', 'blue', 'black', 'cyan']

    

    
    ax = axs['pitch']

    temp = pass_data_clean[pass_data_clean.player_name == player_name]
    top_3_clusters = temp.groupby(['player_name', 'cluster']).count().reset_index().sort_values('team_name', ascending=False).reset_index().cluster[:3].tolist()

    # Iterate through clusters     
    for j in top_3_clusters:
        temp_cluster = temp[temp.cluster == j]

        # In case # of data are less than 10
        if temp_cluster.shape[0] > 10:
            temp_cluster = temp_cluster.sample(10, random_state=42)

        # Visualize the lines of passes             
        lines = pitch.lines(temp_cluster.x, temp_cluster.y, 
                            temp_cluster.end_x, temp_cluster.end_y, 
                            color=colors[j], alpha=0.3, comet=True, ax=ax)

    axs['title'].text(0.5, 0.7, player_name + ' Passing Clusters',
                     va='center', ha='center', fontsize=20)

    #fig.savefig('goalkeepers-clusters.jpeg', bbox_inches='tight')
    fig.show()
    return fig

In [ ]:
def generatePlayerEventMap(player_name):
    
    
    player_role = player_roles_dict[findPlayerRole(player_name)]
    
    if player_role == 'Goalkeeper':
        return getGkBuildUpPassesMap(player_name)
    
    if player_role == 'Center-back':
        return getDefActionMap(player_name)
    
    if player_role == 'Full-back/Wing-back':
        return getCrossMap(player_name)
    
    if player_role == 'Midfielder':
        return getKeyPassPlot(player_name)
    
    if player_role == 'Wide Midfielder/Attacking Midfielder':
        return  getShotMap(player_name)
    
    if player_role == 'Striker':
        return getShotMap(player_name)

In [ ]:
try:
    mydb = connection.connect(host="localhost", database='mydb',user="root", passwd="",use_pure=True)
    query = "SELECT Player.player_name, Position.position_name FROM Event INNER JOIN Player ON Event.player_id=Player.player_id INNER JOIN Position ON Event.position_id=Position.position_id;"
    df_all_event = pd.read_sql(query,mydb)
    mydb.close() #close the connection
except Exception as e:
    mydb.close()
    print(str(e))

def get_max_position_name(group):
    max_count = max(group.position_count)
    for i in range(len(group)):
        row = group.iloc[i]
        if (row.position_count == max_count):
            return row.position_name

top_position_df = df_all_event.groupby(['player_name', 'position_name']).size()
top_position_count_df = top_position_df.reset_index(name='position_count')
max_position_counts = top_position_count_df.groupby(['player_name']).apply(lambda x: max(x.position_count))
player_position_series = top_position_count_df.groupby(['player_name']).apply(get_max_position_name)

player_position_series_df = player_position_series.to_frame(name='position_name')
player_position_series_df = player_position_series_df.reset_index()
player_position_series_df['role_name'] = player_position_series_df['position_name'].apply(lambda x: player_roles_dict[x])
new_dict = player_position_series_df.groupby(['role_name']).apply(lambda x: list(x['player_name'])).to_dict()
new_dict

## GK Plot

In [ ]:
def corresponding_key(val, dictionary):
        for k, v in dictionary.items():
            if val in v:
                return k

In [ ]:
def getPlotGk(player_name, team_name, new_dict):
    
    
    fig = plt.figure(figsize=(10, 7))
    axe = fig.add_subplot(111)
    gk_save_rate = getSaveRate(player_name)
    gk_shots_against = getShotsAgainstPer90(player_name, team_name)
    sp = axe.scatter(gk_save_rate,gk_shots_against, color='r',marker='o')
    axe.text(gk_save_rate * 1.01, gk_shots_against * 1.01, player_name, fontsize=12)
    axe.set_xlabel('Save Rate %')
    axe.set_ylabel('Shots Against p90')
    
    gk_list = [x for x in new_dict[corresponding_key(player_name, new_dict)] if x != player_name] 
    
    try:
        for gk in gk_list:
            mydb = connection.connect(host="localhost", database='mydb',user="root", passwd="",use_pure=True)
            query = "SELECT Player.player_name, Team.team_name FROM Player INNER JOIN Team ON Player.team_id=Team.team_id WHERE Player.player_name=%s;"
            df_player_team = pd.read_sql(query,mydb,params=[gk])
            gk_save_rate = getSaveRate(gk)
            gk_shots_against = getShotsAgainstPer90(gk, df_player_team[df_player_team['player_name'] == gk].team_name.to_list()[0])
            if gk_save_rate==0 and gk_shots_against==0:
                continue
            sp= axe.scatter(gk_save_rate,gk_shots_against, color='k',marker='o')
            mydb.close() #close the connection
    except Exception as e:
            mydb.close()
            print(str(e))
    
    
    axe.set_xlim(0, 100)
    axe.set_ylim(0, 10)
    fig.canvas.draw()
    fig.show()     
    return fig

## CB Plot

In [ ]:
def getPlotCb(player_name, team_name, new_dict):
    
    fig = plt.figure(figsize=(10, 7))
    axe = fig.add_subplot(111)
    cb_x = getTacklesAndInterceptionsPer90(player_name, team_name)
    cb_y = getTackleInterceptionPercentage(player_name)
    sp = axe.scatter(cb_x,cb_y, color='r',marker='o')
    axe.text(cb_x * 1.01, cb_y * 1.01, player_name, fontsize=12)
    axe.set_xlabel('Tackles and Interceptions p90')
    axe.set_ylabel('Tackle and Interception %')
    
    cb_list = [x for x in new_dict[corresponding_key(player_name, new_dict)] if x != player_name] 
    
    try:
        
        for cb in cb_list:
            mydb = connection.connect(host="localhost", database='mydb',user="root", passwd="",use_pure=True)
            query = "SELECT Player.player_name, Team.team_name FROM Player INNER JOIN Team ON Player.team_id=Team.team_id WHERE Player.player_name=%s;"
            df_player_team = pd.read_sql(query,mydb,params=[cb])
            cb_x = getTacklesAndInterceptionsPer90(cb, df_player_team[df_player_team['player_name'] == cb].team_name.to_list()[0])
            cb_y = getTackleInterceptionPercentage(cb)
            if cb_x==0 and cb_y==0:
                continue
            sp= axe.scatter(cb_x,cb_y, color='k',marker='o')
            mydb.close() #close the connection

    
    except Exception as e:
            mydb.close()
            print(str(e))
            
    axe.set_xlim(0, 6)
    axe.set_ylim(0, 100)
    fig.canvas.draw()
    fig.show()     
    return fig

## Plot FBWB

In [ ]:
def getPlotFbWb(player_name, team_name, new_dict):
    
    fig = plt.figure(figsize=(10, 7))
    axe = fig.add_subplot(111)
    fwb_x = getCrossesPer90(player_name, team_name)
    fwb_y = getCrossesPercentage(player_name)
    sp = axe.scatter(fwb_x,fwb_y, color='r',marker='o')
    axe.text(fwb_x * 1.01, fwb_y * 1.01, player_name, fontsize=12)
    axe.set_xlabel('Crosses p90')
    axe.set_ylabel('Crossing %')
    
    fwb_list = [x for x in new_dict[corresponding_key(player_name, new_dict)] if x != player_name] 
    
    try:

        for fwb in fwb_list:
            mydb = connection.connect(host="localhost", database='mydb',user="root", passwd="",use_pure=True)
            query = "SELECT Player.player_name, Team.team_name FROM Player INNER JOIN Team ON Player.team_id=Team.team_id WHERE Player.player_name=%s;"
            df_player_team = pd.read_sql(query,mydb,params=[fwb])
            fwb_x = getCrossesPer90(fwb, df_player_team[df_player_team['player_name'] == fwb].team_name.to_list()[0])
            fwb_y = getCrossesPercentage(fwb)
            if fwb_x==0 and fwb_y==0:
                continue
            sp= axe.scatter(fwb_x,fwb_y, color='k',marker='o')
            mydb.close() #close the connection
    
    except Exception as e:
            mydb.close()
            print(str(e))
            
    axe.set_xlim(0, 6)
    axe.set_ylim(0, 100)
    fig.canvas.draw()
    fig.show()     
    return fig

## Plot Mid

In [ ]:
def getPlotMidf(player_name, team_name, new_dict):
    
    fig = plt.figure(figsize=(10, 7))
    axe = fig.add_subplot(111)
    m_x = getDispossessionsPer90(player_name, team_name)
    m_y = getPassingPercentageUnderPressure(player_name)
    sp = axe.scatter(m_x,m_y, color='r',marker='o')
    axe.text(m_x * 1.01, m_y * 1.01, player_name, fontsize=12)
    axe.set_xlabel('Dispossessions p90')
    axe.set_ylabel('Passing % Under Pressure')
    
    m_list = [x for x in new_dict[corresponding_key(player_name, new_dict)] if x != player_name] 
    
    try:
        for m in m_list:
            mydb = connection.connect(host="localhost", database='mydb',user="root", passwd="",use_pure=True)
            query = "SELECT Player.player_name, Team.team_name FROM Player INNER JOIN Team ON Player.team_id=Team.team_id WHERE Player.player_name=%s;"
            df_player_team = pd.read_sql(query,mydb,params=[m])
            m_x = getDispossessionsPer90(m, df_player_team[df_player_team['player_name'] == m].team_name.to_list()[0])
            m_y = getPassingPercentageUnderPressure(m)
            if m_x==0 and m_y==0:
                continue
            sp= axe.scatter(m_x,m_y, color='k',marker='o')
            mydb.close() #close the connection
    
    except Exception as e:
            mydb.close()
            print(str(e))
    
    
    axe.set_xlim(0.2, 3.5)
    axe.set_ylim(40, 100)
    fig.canvas.draw()
    fig.show() 
    return fig

## Plot WMAM

In [ ]:
def getPlotWmAm(player_name, team_name, new_dict):
    
    fig = plt.figure(figsize=(10, 7))
    axe = fig.add_subplot(111)
    wam_x = getKeyPassesPer90(player_name, team_name)
    wam_y = getxGValuePer90(player_name, team_name)
    sp = axe.scatter(wam_x,wam_y, color='r',marker='o')
    axe.text(wam_x * 1.01, wam_y * 1.01, player_name, fontsize=12)
    axe.set_xlabel('Key Passes p90')
    axe.set_ylabel('xG p90')
    
    wam_list = [x for x in new_dict[corresponding_key(player_name, new_dict)] if x != player_name] 
    
    try:
        for wam in wam_list:
            mydb = connection.connect(host="localhost", database='mydb',user="root", passwd="",use_pure=True)
            query = "SELECT Player.player_name, Team.team_name FROM Player INNER JOIN Team ON Player.team_id=Team.team_id WHERE Player.player_name=%s;"
            df_player_team = pd.read_sql(query,mydb,params=[wam])
            wam_x = getKeyPassesPer90(wam, df_player_team[df_player_team['player_name'] == wam].team_name.to_list()[0])
            wam_y = getxGValuePer90(wam, df_player_team[df_player_team['player_name'] == wam].team_name.to_list()[0])
            if wam_x==0 and wam_y==0:
                continue
            sp= axe.scatter(wam_x,wam_y, color='k',marker='o')
            mydb.close() #close the connection
    
    except Exception as e:
            mydb.close()
            print(str(e))
    
    
    axe.set_xlim(0, 3.5)
    axe.set_ylim(0, 1)
    fig.canvas.draw()
    fig.show()    
    return fig

## Plot Striker

In [ ]:
def getPlotStr(player_name, team_name, new_dict):
    
    fig = plt.figure(figsize=(10, 7))
    axe = fig.add_subplot(111)
    s_x = getNumberOfGoalsPer90(player_name, team_name)
    s_y = getxGValuePer90(player_name, team_name)
    sp = axe.scatter(s_x,s_y, color='r',marker='o')
    axe.text(s_x * 1.01, s_y * 1.01, player_name, fontsize=12)
    axe.set_xlabel('Goals Per Game')
    axe.set_ylabel('xG Per Game')
    
    s_list = [x for x in new_dict[corresponding_key(player_name, new_dict)] if x != player_name] 
    
    try:
        for s in s_list:
            mydb = connection.connect(host="localhost", database='mydb',user="root", passwd="",use_pure=True)
            query = "SELECT Player.player_name, Team.team_name FROM Player INNER JOIN Team ON Player.team_id=Team.team_id WHERE Player.player_name=%s;"
            df_player_team = pd.read_sql(query,mydb,params=[s])
            s_x = getNumberOfGoalsPer90(s, df_player_team[df_player_team['player_name'] == s].team_name.to_list()[0])
            s_y = getxGValuePer90(s, df_player_team[df_player_team['player_name'] == s].team_name.to_list()[0])
            if s_x==0 and s_y==0:
                continue
            sp= axe.scatter(s_x,s_y, color='k',marker='o')
            mydb.close() #close the connection
    
    except Exception as e:
            mydb.close()
            print(str(e))
    
    
    axe.set_xlim(0, 1)
    axe.set_ylim(0, 1.0)
    fig.canvas.draw()
    fig.show()     
    return fig

In [ ]:
def generateScatterPlot(player_name, team_name, new_dict):
    
    
    player_role = player_roles_dict[findPlayerRole(player_name)]
    
    if player_role == 'Goalkeeper':
        return getPlotGk(player_name, team_name, new_dict)
    
    if player_role == 'Center-back':
        return getPlotCb(player_name, team_name, new_dict)
    
    if player_role == 'Full-back/Wing-back':
        return getPlotFbWb(player_name, team_name, new_dict)
    
    if player_role == 'Midfielder':
        return getPlotMidf(player_name, team_name, new_dict)
    
    if player_role == 'Wide Midfielder/Attacking Midfielder':
        return  getPlotWmAm(player_name, team_name, new_dict)
    
    if player_role == 'Striker':
        return getPlotStr(player_name, team_name, new_dict)

## TABLE

In [ ]:
import plotly.graph_objects as go
def getTable(player_name, team_name):
    
    if findPlayerRole(player_name) == 'Goalkeeper':
        fig = go.Figure(data=[go.Table(header=dict(values=['GP', 'G', 'A', 'GC', 'YC', 'RC']),
                         cells=dict(values=[getNumberOfPresences(player_name, team_name), getNumberOfGoals(player_name), getNumberOfAssists(player_name), getGoalsConceded(player_name), getNumberOfYellowCards(player_name), getNumberOfRedCards(player_name)]))])
        fig.update_layout(
            autosize=False,
            width=400,
            height=250)
        fig.show()
        return fig
    else:
        fig = go.Figure(data=[go.Table(header=dict(values=['GP', 'G', 'A', 'YC', 'RC']),
                         cells=dict(values=[getNumberOfPresences(player_name, team_name), getNumberOfGoals(player_name), getNumberOfAssists(player_name), getNumberOfYellowCards(player_name), getNumberOfRedCards(player_name)]))])
        fig.update_layout(
            autosize=False,
            width=400,
            height=250)
        fig.show()
        return fig

In [ ]:
df_all_lineups = pd.DataFrame()

try:
    mydb = connection.connect(host="localhost", database='mydb',user="root", passwd="",use_pure=True)
    query = "SELECT Player.player_id, Player.player_name, Team.team_id, Team.team_name FROM Lineup INNER JOIN Player ON Lineup.player_id=Player.player_id INNER JOIN Team ON Lineup.team_id=Team.team_id;"
    df_all_lineups = pd.read_sql(query,mydb)
    mydb.close() #close the connection
    
except Exception as e:
    mydb.close()
    print(str(e))
    
df_all_lineups[['player_id', 'player_name', 'team_id', 'team_name']].drop_duplicates()
team_name_player_name_dict = df_all_lineups.groupby(['team_name'])['player_name'].apply(lambda grp: list(grp.value_counts().index)).to_dict()
#team_name_player_name_dict

In [ ]:
import param
import hvplot.pandas
import plotly.graph_objects as go

class PlayerDashboard(param.Parameterized):
    
    team_selector = param.Selector(default='Argentina', objects=list(team_name_player_name_dict.keys()))
    player_selector = param.Selector(default='Lionel Andrés Messi Cuccittini', objects=team_name_player_name_dict['Argentina'])
    match_dict_home = match_dict_home
    match_name_dict = match_name_dict
    team_name_player_name_dict = team_name_player_name_dict
    new_dict = new_dict
    player_roles_dict = player_roles_dict
    match_selector = param.Selector(default='Argentina - Saudi Arabia', objects= list(map(lambda x: x['match_name'], match_dict_home['Argentina'])))
    compare_with = param.Selector(default='Select a player', objects=['Select a player'])
    
    @param.depends('team_selector', watch=True)
    def _update_players(self):
        self.param['player_selector'].objects = team_name_player_name_dict[self.team_selector]
        self.player_selector = team_name_player_name_dict[self.team_selector][0]
    
    def heatmap_view(self):
        return generatePlayerHeatMap(self.player_selector, self.match_selector, self.match_name_dict)
    
    def radar_view(self):
        new_list = ['Select a player']
        new_list.extend([x for x in new_dict[corresponding_key(self.player_selector, new_dict)] if x != self.player_selector])
        self.param['compare_with'].objects = new_list
        self.param['compare_with'].default = new_list[0]
        return generatePlayerRadar(self.player_selector, self.team_selector, self.compare_with, corresponding_key(self.compare_with, team_name_player_name_dict))
    
    def table_view(self):
        return getTable(self.player_selector, self.team_selector)
    
    def map_view(self):
        return generatePlayerEventMap(self.player_selector)
    
    def plot_view(self):
        return generateScatterPlot(self.player_selector, self.team_selector, self.new_dict)
    
    @param.depends('team_selector', watch=True)
    def _update_matches(self):
        self.param['match_selector'].objects = list(map(lambda x: x['match_name'], match_dict_home[self.team_selector]))
        self.match_selector = list(map(lambda x: x['match_name'], match_dict_home[self.team_selector]))[0]
        
        
player_dashboard = PlayerDashboard()

In [ ]:
title = '#World Cup 2022 Player Dashboard'
logo_statsbomb = pn.panel('/Users/bernardo/Desktop/open-data-master/img/SB - Icon Lockup - Colour positive.png', width=350, align='center')
header_box = pn.WidgetBox(title, pn.widgets.Select.from_param(player_dashboard.param.team_selector, sizing_mode='stretch_width'), pn.widgets.Select.from_param(player_dashboard.param.player_selector, sizing_mode='stretch_width'), pn.Column(player_dashboard.table_view, sizing_mode='stretch_both'), pn.layout.Spacer(margin=150), logo_statsbomb, pn.layout.Spacer(sizing_mode='stretch_height'), width=400, align="start")
heatmap_box = pn.Column('##Heatmap', pn.Column(pn.widgets.Select.from_param(player_dashboard.param.match_selector, sizing_mode='stretch_width'), player_dashboard.heatmap_view, pn.layout.Spacer(sizing_mode='scale_both')), margin=(20,50))
radar_box = pn.Column('##Radar', pn.Column(player_dashboard.param.compare_with , player_dashboard.radar_view), sizing_mode='stretch_both')
scatter_box = pn.Column('##Scatter Plot', player_dashboard.plot_view)
map_box = pn.Column('##Map', player_dashboard.map_view)
plots_box = pn.WidgetBox(pn.Column(pn.Row(heatmap_box, radar_box), pn.Row(scatter_box, map_box), width=1400), width=1400, sizing_mode='stretch_both')
# Dashboard
dashboard = pn.Row(header_box, pn.Spacer(sizing_mode='stretch_height'), plots_box, pn.Spacer(sizing_mode='stretch_height'))
dashboard.show()